In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),#undef)

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("b2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
x = MechanismState{Float64}(mechanism);

In [6]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  RigidBodyDynamics.Joint… => [0.4178499277227168,0.10229415024901356,0.2242149…
  RigidBodyDynamics.Joint… => [0.5179766326522544]

In [7]:
num_positions(x)

8

In [8]:
num_velocities(x)

7

In [9]:
cache = MechanismStateCache(mechanism, x);

In [10]:
relative_transform(cache, joint2.frameAfter, body1.frame)

RigidBodyDynamics.Transform3D{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("body1"),-0.9599046518939521 + 0.25384857822659695im + 0.0050783968649733jm + 0.11881989938544585km,Vec(0.4502791911670121,0.7045075769912708,0.47617947361085666))

In [11]:
center_of_mass(mechanism, cache)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(-1.7369891398601902,-0.09983057607092022,4.3919200983471445))

In [12]:
mass(mechanism)

0.22626327399700208

In [13]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root(mechanism).frame)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(-2.068664326784159,-0.18878756522418305,2.970086680693991))

In [14]:
relative_twist(cache, body2, body1)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(-0.032032466932445935,-0.24248161004257252,0.2698750770835567),Vec(0.6692423415833856,0.4631417412650504,0.49556572515595665))

In [15]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.0,0.0,0.0),Vec(0.0,0.0,0.0))

In [16]:
geometric_jacobian(cache, path(mechanism, body2, world))

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("before_1"),RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
 -0.0618415  -0.609381  -0.189337  …  -0.0       -0.0       -0.0      
 -0.468132   -0.688984  -0.354102     -0.0       -0.0       -0.0      
 -0.521018   -0.392371  -0.91584      -0.0       -0.0       -0.0      
 -1.29203    -0.633077  -0.452048     -0.609381  -0.189337  -0.769939 
 -0.894136   -0.218791  -1.07429      -0.688984  -0.354102  -0.632387 
 -0.956734   -0.59903   -0.32191   …  -0.392371  -0.91584   -0.0853329)